In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install fastai=="1.0.61"
!pip install torchsummary

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from google.colab import drive
drive.mount('/content/drive')
from glob import glob
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchsummary import summary
from fastai.metrics import accuracy
from fastai.vision import *
import numpy as np
!pip install fft-conv-pytorch
from fft_conv_pytorch import fft_conv, FFTConv2d

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
# from fastai.vision.data import normalize
path = "/content/drive/MyDrive/Dataset/covid-19_dataset_grayscale"
np.random.seed(41)
defaults = SimpleNamespace(cmap='L', return_fig=False, silent=False)
data = ImageDataBunch.from_folder(path, train="Train", valid ="Valid", valid_pct=0.2,ds_tfms=get_transforms(), size=(512,512), bs=16, num_workers=4)
for itemList in ["train_dl", "valid_dl", "fix_dl", "test_dl"]:
    itemList = getattr(data, itemList)
    if itemList: itemList.x.convert_mode = "L"

In [7]:
print (data.one_batch()[0].shape)
print (data.classes, data.c, len(data.train_ds), len(data.valid_ds))
print (data.train_ds.classes)
print (data.valid_ds.classes)

torch.Size([16, 1, 512, 512])
['covid-19', 'no_findings', 'pneumonia'] 3 900 225
['covid-19', 'no_findings', 'pneumonia']
['covid-19', 'no_findings', 'pneumonia']


In [ ]:
def fft_conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        FFTConv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def conv_block(ni, nf, size=3, stride=1):
    for_pad = lambda s: s if s > 2 else 3
    return nn.Sequential(
        nn.Conv2d(ni, nf, kernel_size=size, stride=stride,
                  padding=(for_pad(size) - 1)//2, bias=False), 
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(negative_slope=0.1, inplace=True)  
    )

def triple_conv(ni, nf):
    return nn.Sequential(
        conv_block(ni, nf),
        conv_block(nf, ni, size=1),  
        conv_block(ni, nf)
    )
def fft_triple_conv(ni, nf,size):
    return nn.Sequential(
        fft_conv_block(ni, nf,size),
        conv_block(nf, ni, size=1),  
        fft_conv_block(ni, nf, size)
    )
def maxpooling():
    return nn.MaxPool2d(2, stride=2)

In [ ]:
fft_model3 = nn.Sequential(
    fft_conv_block(1,8,21),
    maxpooling(),
    fft_conv_block(8,16,21),
    maxpooling(),
    triple_conv(16,32),
    maxpooling(),
    triple_conv(32, 64),
    maxpooling(),
    triple_conv(64, 128),
    maxpooling(),
    triple_conv(128, 256),
    conv_block(256, 128, size=1),
    conv_block(128, 256),
    conv_block(256,3),
    nn.Flatten(),
    nn.Linear(1323, 3)
)
fft_learn3 = Learner(data, fft_model3, loss_func = nn.CrossEntropyLoss(), metrics=accuracy)
print(fft_learn3.summary())
fft_learn3.fit_one_cycle(100, max_lr=5e-3)
probs,targets = fft_learn3.get_preds(ds_type=DatasetType.Valid) 
accuracy(probs,targets)
# fft_learn3.recorder.plot_losses()

Sequential
Layer (type)         Output Shape         Param #    Trainable 
_FFTConv             [8, 512, 512]        3,528      True      
______________________________________________________________________
BatchNorm2d          [8, 512, 512]        16         True      
______________________________________________________________________
LeakyReLU            [8, 512, 512]        0          False     
______________________________________________________________________
MaxPool2d            [8, 256, 256]        0          False     
______________________________________________________________________
_FFTConv             [16, 256, 256]       56,448     True      
______________________________________________________________________
BatchNorm2d          [16, 256, 256]       32         True      
______________________________________________________________________
LeakyReLU            [16, 256, 256]       0          False     
___________________________________________________

epoch,train_loss,valid_loss,accuracy,time
0,0.811395,0.673413,0.720000,00:35
1,0.724898,0.544044,0.773333,00:34
2,0.664768,0.779562,0.648889,00:34
3,0.644528,0.490324,0.786667,00:34
4,0.660232,0.576728,0.782222,00:35
5,0.651165,1.402030,0.497778,00:35
6,0.621650,0.704282,0.680000,00:34
7,0.640296,0.730283,0.702222,00:34
8,0.678760,0.713139,0.702222,00:34
9,0.713887,1.970317,0.551111,00:34


tensor(0.8667)